## 1. 문서 단어 행렬 (DTM) 정의 및 수동 생성 방법

**문서 단어 행렬 (DTM)** 이란 여러 문서에서 등장하는 각 단어들의 빈도를 행렬로 표현한 것입니다. 각 행은 문서를, 각 열은 단어를 나타내며, 행렬의 값은 해당 문서에서 해당 단어가 나타난 횟수를 의미합니다.

**CountVectorizer를 사용하지 않고 DTM을 만드는 방법 (Python):**

```python
from collections import defaultdict

def create_dtm(documents):
    """
    문서 리스트를 입력받아 DTM을 생성하는 함수.

    Args:
        documents: 문서 리스트 (예: ["나는 영화가 좋다", "영화는 재미있다"])

    Returns:
        DTM (dict), 단어 사전 (dict)
    """
    
    # 1. 단어 사전 만들기
    word_dict = defaultdict(lambda: len(word_dict))
    
    
    # defaultdict는 기본적으로 딕셔너리와 거의 비슷하게 동작하지만, 한 가지 다른 점은
    # defaultdict에 키(key)가 없는 경우에도 에러를 발생시키지 않고,
    # 미리 지정해둔 기본값(default value)을 반환한다는 점입니다.
    
    # defaultdict(lambda: len(word_dict))는 새로운 키가 추가될 때마다
    # word_dict의 현재 길이를 기본값으로 사용하는 defaultdict를 생성합니다.
    # 이는 단어에 고유한 ID를 할당하는 데 사용됩니다.
    
    
    # 2. DTM 생성
    dtm = []
    for doc in documents:
        term_frequency = {}  # 문서 내 단어 빈도
        for word in doc.split():
            term_frequency[word_dict[word]] = term_frequency.get(word_dict[word], 0) + 1  
            # defaultdict를 사용하면, 딕셔너리에 해당 키가 없는 경우,
            # 지정된 기본값을 반환합니다.
            # word_dict[word]를 사용하여 단어에 대한 고유 ID를 가져오고,
            # 이 ID를 키로 사용하여 term_frequency 딕셔너리에 접근합니다.
            # 만약 해당 ID가 term_frequency에 없다면, .get(word_dict[word], 0)는 기본값 0을 반환하고,
            # 여기에 1을 더하여 빈도를 계산합니다.
            
            
        dtm.append(term_frequency) # dtm에 추가
        

    return dtm, word_dict

# 예시 문서
documents = [
    "나는 영화가 좋다",
    "영화는 재미있다",
    "나는 영화가 싫다"
]

# DTM 생성
dtm, word_dict = create_dtm(documents)

print("DTM:", dtm)
print("단어 사전:", word_dict)

```

**설명:**

1.  **단어 사전 만들기:**
    *   `word_dict`: `defaultdict`를 사용하여, 단어가 사전에 없으면 자동으로 새로운 ID를 할당합니다.
    *   `doc.split()`: 문서를 공백 기준으로 나누어 단어 리스트를 만듭니다.
    *   `word_dict[word]`: 단어에 대한 고유한 숫자 ID를 할당하거나 가져옵니다.
    *   `term_frequency`: 각 문서에서 단어의 빈도를 {단어 ID: 빈도} 형태로 저장합니다.
    *   `term_frequency.get(word_dict[word], 0) + 1`: 단어의 빈도를 1 증가시킵니다. 해당 단어가 처음 나타나면 0(기본값) + 1이 됩니다.

2.  **DTM 생성:**
    *   `dtm`: DTM을 리스트 형태로 만듭니다. 각 행은 문서, 각 열은 단어 ID를 나타냅니다. (단어의 인덱스를 열로 표현)
    *   `dtm.append(term_frequency)`: 각 문서의 단어 빈도 딕셔너리를 DTM에 추가합니다.



## 2. DTM을 이용한 영화별 상관성 계산

DTM을 이용하여 영화 간의 상관성을 계산하려면, 각 영화를 DTM의 행(row)으로 간주하고, 각 행 벡터 간의 유사도를 측정해야 합니다. 일반적으로 코사인 유사도(Cosine Similarity)가 많이 사용됩니다.

**코사인 유사도:**

두 벡터 간의 각도의 코사인 값을 이용하여 유사도를 측정합니다. 값이 1에 가까울수록 유사도가 높고, 0에 가까울수록 유사도가 낮습니다.

```python
import numpy as np

def cosine_similarity(vector1, vector2):
    """
    두 벡터의 코사인 유사도를 계산하는 함수.

    Args:
        vector1: 첫 번째 벡터 (NumPy 배열)
        vector2: 두 번째 벡터 (NumPy 배열)

    Returns:
        코사인 유사도 (float)
    """
    dot_product = np.dot(vector1, vector2)
    norm_vector1 = np.linalg.norm(vector1)
    norm_vector2 = np.linalg.norm(vector2)

    if norm_vector1 == 0 or norm_vector2 == 0:
        return 0  # 분모가 0이 되는 경우 방지
    else:
        return dot_product / (norm_vector1 * norm_vector2)

def calculate_similarity_matrix(dtm, word_dict):
    """
    DTM을 기반으로 영화 간 유사도 행렬을 계산하는 함수.

    Args:
        dtm: DTM (list of dicts)
        word_dict: 단어 사전 (dict)

    Returns:
        유사도 행렬 (NumPy 배열)
    """
        
    num_movies = len(dtm) # 영화 개수
    
    # DTM 희소 행렬을 밀집 행렬(NumPy 배열)로 변환
    num_words = len(word_dict) # 단어 개수
    dense_dtm = np.zeros((num_movies, num_words)) # 0으로 채워진 행렬
    for i, doc_freq in enumerate(dtm):
        for word_id, freq in doc_freq.items():
            dense_dtm[i, word_id] = freq # 문서 내 단어 빈도를 밀집 행렬에 채워 넣음

    # 영화 간 유사도 행렬 계산
    similarity_matrix = np.zeros((num_movies, num_movies))
    for i in range(num_movies):
        for j in range(i, num_movies):  # 대칭 행렬이므로 상삼각 행렬만 계산
            similarity = cosine_similarity(dense_dtm[i], dense_dtm[j]) # i번째 영화와 j번째 영화의 코사인 유사도 계산
            similarity_matrix[i, j] = similarity
            similarity_matrix[j, i] = similarity  # 대칭 행렬이므로 반대쪽도 채움
            
    return similarity_matrix

# 영화 간 유사도 행렬 계산
similarity_matrix = calculate_similarity_matrix(dtm, word_dict)
print("유사도 행렬:\n", similarity_matrix)
```

**설명:**

1.  **`create_dense_dtm` 함수:**
    *   DTM(리스트)를 NumPy 배열(밀집 행렬)로 변환합니다. 이렇게 해야 코사인 유사도 계산이 용이합니다.
    *   `np.zeros((len(dtm), len(word_dict)))`: 0으로 채워진 행렬을 만듭니다.

2.  **`calculate_similarity_matrix` 함수:**
    *   `cosine_similarity` 함수를 사용하여 각 영화 쌍의 코사인 유사도를 계산합니다.
    *   `similarity_matrix`: 모든 영화 쌍 간의 유사도를 저장하는 2차원 NumPy 배열입니다.



## 3. 상관성에 따른 영화 추천/비추천 목록 추출

```python
def recommend_movies(similarity_matrix, movie_index, top_n=2):
    """
    주어진 영화와 유사도가 높은/낮은 영화 목록을 추천/비추천하는 함수.

    Args:
        similarity_matrix: 영화 간 유사도 행렬 (NumPy 배열)
        movie_index: 기준 영화의 인덱스 (int)
        top_n: 추천/비추천할 영화 수 (int)

    Returns:
        추천 영화 인덱스 리스트, 비추천 영화 인덱스 리스트
    """

    # 해당 영화와의 유사도 (자기 자신 제외)
    movie_similarities = similarity_matrix[movie_index]
    movie_similarities[movie_index] = -1  # 자기 자신과의 유사도는 -1로 설정하여 제외

    # 유사도가 높은 순으로 정렬된 인덱스 (추천)
    recommended_indices = np.argsort(movie_similarities)[::-1][:top_n]  # 내림차순 정렬 후 top_n개 선택

    # 유사도가 낮은 순으로 정렬된 인덱스 (비추천)
    not_recommended_indices = np.argsort(movie_similarities)[:top_n]  # 오름차순 정렬 후 top_n개 선택

    return recommended_indices, not_recommended_indices

# 예시: 0번 영화를 기준으로 추천/비추천
movie_index = 0
recommended, not_recommended = recommend_movies(similarity_matrix, movie_index)

print(f"{movie_index}번 영화와 유사한 영화:", recommended)
print(f"{movie_index}번 영화와 다른 영화:", not_recommended)
```

**설명:**

1.  `recommend_movies` 함수:
    *   `movie_similarities`: 특정 영화(`movie_index`)와 다른 모든 영화 간의 유사도 값을 담은 배열입니다.
    *   `np.argsort()`: 배열을 오름차순으로 정렬하고, 정렬된 인덱스를 반환합니다.
    *   `[::-1]`:  `np.argsort()`의 결과를 뒤집어 내림차순으로 만듭니다.
    *   `[:top_n]`: 상위 `top_n`개의 인덱스만 선택합니다.




## 4. 피어슨 상관계수와 활용

**피어슨 상관계수 (Pearson Correlation Coefficient):**

두 변수 간의 선형 상관 관계를 측정하는 통계적 지표입니다. -1에서 1 사이의 값을 가지며,

*   1: 완벽한 양의 선형 상관관계
*   0: 선형 상관관계 없음
*   -1: 완벽한 음의 선형 상관관계

**위의 질문에서 피어슨 상관계수의 활용:**

위의 예제에서는 코사인 유사도를 사용했지만, 피어슨 상관계수를 사용하여 영화 간의 상관성을 계산할 수도 있습니다.  DTM의 각 행(영화)을 하나의 변수로 보고, 각 열(단어)을 관측값으로 생각하면, 두 영화 벡터 간의 피어슨 상관계수를 계산할 수 있습니다.

**NumPy를 이용한 피어슨 상관계수 계산:**

```python
import numpy as np

# DTM을 NumPy 배열로 변환 (위의 create_dense_dtm 함수 참고)
dense_dtm = np.zeros((len(dtm), len(word_dict)))
for i, doc_freq in enumerate(dtm):
    for word_id, freq in doc_freq.items():
        dense_dtm[i, word_id] = freq
        
# 피어슨 상관계수 행렬 계산
pearson_correlation_matrix = np.corrcoef(dense_dtm)

print("피어슨 상관계수 행렬:\n", pearson_correlation_matrix)

# 추천/비추천 목록 추출 (recommend_movies 함수 사용 가능)
movie_index = 0
recommended, not_recommended = recommend_movies(pearson_correlation_matrix, movie_index)

print(f"{movie_index}번 영화와 유사한 영화 (피어슨):", recommended)
print(f"{movie_index}번 영화와 다른 영화 (피어슨):", not_recommended)

```

**`np.corrcoef()` 함수:**

*   입력으로 주어진 행렬의 각 행(row) 간의 피어슨 상관계수를 계산하여 상관계수 행렬을 반환합니다.  이 행렬은 대칭 행렬이며, 주 대각선은 항상 1입니다 (자기 자신과의 상관관계).

**코사인 유사도 vs. 피어슨 상관계수:**

*   **코사인 유사도:**  두 벡터의 방향(각도)에 초점을 맞춥니다. 벡터의 크기(길이)는 무시합니다.  텍스트 데이터에서 단어 빈도의 절대적인 차이보다는 단어 사용 패턴의 유사성을 보는 데 유용합니다.
*   **피어슨 상관계수:** 두 벡터의 선형 관계를 측정합니다.  벡터의 크기와 방향을 모두 고려합니다.  데이터의 평균을 중심으로 한 변화 패턴의 유사성을 측정하는 데 유용합니다.

어떤 유사도/상관성 지표를 사용할지는 데이터의 특성과 분석 목적에 따라 결정해야 합니다.  텍스트 데이터의 경우에는 코사인 유사도가 일반적으로 더 좋은 성능을 보이는 경향이 있습니다.